In [ ]:
pip install keybert sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_csv("nlp_merged_reviews.csv")


In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT(model="all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def extract_benefit_keywords(text):
    prompted_text = "benefits advantages positives good features " + text

    keywords = kw_model.extract_keywords(
        prompted_text,                # <-- FIXED (first positional argument)
        keyphrase_ngram_range=(1, 2),
        stop_words='english',
        top_n=20
    )
    return [k[0] for k in keywords]



In [ ]:
def extract_disadvantage_keywords(text):
    prompted_text = "issues problems negatives disadvantages complaints bad " + text

    keywords = kw_model.extract_keywords(
        prompted_text,                 # <-- FIXED
        keyphrase_ngram_range=(1, 2),
        stop_words='english',
        top_n=20
    )
    return [k[0] for k in keywords]



In [ ]:
product_data = {}

for i, row in df.iterrows():
    product = row["product_name"]
    text = row["merged_reviews"]

    benefits = extract_benefit_keywords(text)
    disadvantages = extract_disadvantage_keywords(text)

    product_data[product] = {
        "benefits": benefits,
        "disadvantages": disadvantages
    }


In [ ]:
product_data["Fenty Pro Filt'r Soft Matte Foundation"]


{'benefits': ['foundation recommend',
  'recommended foundation',
  'skin foundation',
  'foundation oily',
  'foundation good',
  'foundation better',
  'foundation best',
  'best foundation',
  'buy foundation',
  'foundation routine',
  'foundation amazing',
  'foundation great',
  'foundation look',
  'color foundation',
  'buying foundation',
  'foundation looks',
  'foundation face',
  'blend foundation',
  'using foundation',
  'foundation patchy'],
 'disadvantages': ['foundation recommend',
  'foundation oily',
  'recommended foundation',
  'skin foundation',
  'buy foundation',
  'foundation best',
  'foundation good',
  'foundation patchy',
  'best foundation',
  'foundation better',
  'foundation routine',
  'foundation issue',
  'foundation oxidizes',
  'buying foundation',
  'color foundation',
  'blend foundation',
  'foundation face',
  'foundation look',
  'foundation amazing',
  'wear foundation']}

In [ ]:
product_terms = [
    "foundation", "foundations",
    "cream", "moisturizer",
    "product", "makeup"
]


In [ ]:
def clean_text(text):
    text = text.lower()
    for t in product_terms:
        text = text.replace(t, "")
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [ ]:
def extract_benefits(text):
    prompted = (
        "hydrating glowy smooth natural lightweight gentle bright "
        "benefits positives great amazing love nice soft "
        + clean_text(text)
    )

    keywords = kw_model.extract_keywords(
        prompted,
        keyphrase_ngram_range=(1,1),
        stop_words='english',
        use_maxsum=True,
        top_n=20
    )

    return [k[0] for k in keywords if len(k[0]) > 2]


In [ ]:
def extract_disadvantages(text):
    prompted = (
        "drying cakey oxidizes heavy patchy breakouts irritation "
        "negatives bad complaints issues problems concerns "
        + clean_text(text)
    )

    keywords = kw_model.extract_keywords(
        prompted,
        keyphrase_ngram_range=(1,1),
        stop_words='english',
        use_maxsum=True,
        top_n=20
    )

    return [k[0] for k in keywords if len(k[0]) > 2]


In [ ]:
product_keywords = {}

for _, row in df.iterrows():
    product = row["product_name"]
    text = row["merged_reviews"]

    benefits = extract_benefits(text)
    disadvantages = extract_disadvantages(text)

    product_keywords[product] = {
        "benefits": benefits,
        "disadvantages": disadvantages
    }


In [ ]:
product_keywords["Fenty Pro Filt'r Soft Matte Foundation"]


{'benefits': ['pimples',
  'fragrance',
  'hydrated',
  'serums',
  'pimple',
  'oily',
  'skin',
  'hydrating',
  'moist',
  'glowy',
  'blemishes',
  'tan',
  'rosacea',
  'mattifying',
  'acne',
  'wrinkle',
  'skincare',
  'sunscreen',
  'complexion',
  'exfoliate'],
 'disadvantages': ['oxidize',
  'fragrance',
  'mattifying',
  'skin',
  'rosacea',
  'hyaluronic',
  'dry',
  'tan',
  'drying',
  'moist',
  'pimples',
  'pimple',
  'oily',
  'sunscreen',
  'skincare',
  'complexion',
  'blemishes',
  'wrinkle',
  'acne',
  'exfoliate']}

In [ ]:
from transformers import pipeline

sentiment = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def filter_positive(words):
    out = []
    for w in words:
        result = sentiment(w)[0]
        if result['label'] == 'POSITIVE':
            out.append(w)
    return out

def filter_negative(words):
    out = []
    for w in words:
        result = sentiment(w)[0]
        if result['label'] == 'NEGATIVE':
            out.append(w)
    return out


In [ ]:
raw_benefits = product_keywords[product]["benefits"]
raw_disadvantages = product_keywords[product]["disadvantages"]

final_benefits = filter_positive(raw_benefits)
final_disadvantages = filter_negative(raw_disadvantages)


In [ ]:
product = "Lancôme Rénergie H.P.N. 300-Peptide Cream"

raw_benefits = product_keywords[product]["benefits"]
raw_disadvantages = product_keywords[product]["disadvantages"]

final_benefits = filter_positive(raw_benefits)
final_disadvantages = filter_negative(raw_disadvantages)

print("BENEFITS:")
print(final_benefits)

print("\nDISADVANTAGES:")
print(final_disadvantages)


BENEFITS:
['skincares', 'rosacea', 'sunscreens', 'skincare', 'lauder', 'glowy', 'sunscreen', 'lotion', 'complexion', 'fragrance', 'moisturization', 'moisturizes', 'moisturizing']

DISADVANTAGES:
['serum', 'oily', 'spf50', 'spf25', 'acne', 'wrinkles', 'spf']
